In [5]:
import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
alter table fin.revenue_est_2025 drop column id;
alter table fin.revenue_est_2025_backup rename to revenue_est_2025;
alter table fin.revenue_est_2025_old add column id serial;
alter table fin.revenue_est_2025_old add primary key (id);
create table fin.revenue_est_2025_backup as (
	select id, company, date_dt, estimate_date, frc, est_amount, hcl_amount, contr_amount
	from fin.revenue_est_2025
);
drop table fin.revenue_est_2025_old;

alter table fin.revenue_fact rename to revenue_fact_old;
alter table fin.revenue_fact_old add column id serial;
alter table fin.revenue_fact_old add primary key (id);
create table fin.revenue_fact_backup as (
	select id, date_dt, c_agent, contract, doc, division, frc, nom_g, div_frc, nom_frc, amount
	from fin.revenue_fact
);



drop table fin.revenue_fact_old;


alter table fin.revenue_plan_2025 rename to revenue_plan_2025_old;
alter table fin.revenue_plan_2025_old add column id serial;
alter table fin.revenue_plan_2025_old add primary key (id);
create table fin.revenue_plan_2025_backup as (
	select id, company, date_dt, frc, amount
	from fin.revenue_plan_2025
);
drop table fin.revenue_plan_2025_old;

truncate table fin.revenue_est_2025;

create sequence fin.fact_id;
create sequence fin.plan_id;
create sequence fin.est_id;

drop sequence fin.revenue_fact_seq;

alter table fin.revenue_fact  drop column id;
alter table fin.revenue_est_2025  drop column id;
alter table fin.revenue_plan_2025  drop column id;
alter table fin.revenue_fact_old rename to revenue_fact;

truncate table fin.revenue_est_2025;

In [145]:
class MonthlyUpdater:
    def __init__(self):
        
        self.__get_engine()
        
    def __load_dotenv(self):
        load_dotenv()
        return {
            'user': os.getenv("DB_USER"),
            'password': os.getenv("DB_PASSWORD"),
            'host': 'localhost',
            'port': os.getenv("DB_PORT"),
            'db_name': os.getenv("DB_NAME"),
            'schema': 'fin'
        }

    def __get_engine(self):
        conf = self.__load_dotenv()
        self.__engine = create_engine(f"postgresql+psycopg2://{conf['user']}:{conf['password']}@{conf['host']}:{conf['port']}/{conf['db_name']}")

    def __read_sql_query(self, query):
        with self.__engine.begin() as con:
            return pd.read_sql_query(query, con)

    def __get_target_df(self):
        # c_month = datetime.now().month 
        c_month = 9
        c_year = datetime.now().year
        query = f"""
            select * 
            from fin.revenue_est_2025 rev
            where date_part('month', rev.estimate_date) = {c_month - 1}
            and date_part('year', rev.estimate_date) = {c_year}
            and date_part('month', rev.date_dt) >= {c_month}
        """
        data = self.__read_sql_query(query)
        return (
            data
            .assign(estimate_date = lambda x: x['estimate_date'] + relativedelta(months=1),
                     date_dt = lambda x: x['date_dt'].astype("datetime64[ns]").dt.strftime("%Y-%m-%d"),
                    est_amount = lambda x: x['est_amount'].astype('string').apply(lambda x: str(x)).str.replace('<NA>', 'NULL'),
                    hcl_amount = lambda x: x['hcl_amount'].astype('string').apply(lambda x: str(x)).str.replace('<NA>', 'NULL'),
                    contr_amount = lambda x: x['contr_amount'].astype('string').apply(lambda x: str(x)).str.replace('<NA>', 'NULL')
                   )
            .assign(estimate_date = lambda x: x['estimate_date'].astype("datetime64[ns]").dt.strftime("%Y-%m-%d"))
        )

    def __get_query(self):
        df = self.__get_target_df()
        l_q = list(zip(df['company'], df['date_dt'], df['estimate_date'], df['frc'], df['est_amount'], df['hcl_amount'], df['contr_amount']))
        res_list = []
        for x, y, z, k, l, m, n in l_q:
            res_list.append(f"('{x}', '{y}', '{z}', '{k}', {l}, {m}, {n})")
        return f"""insert into fin.revenue_est_2025 (company, date_dt, estimate_date, frc, est_amount, hcl_amount, contr_amount) values {', '.join(res_list)}"""

    def execute_query(self):
        query = self.__get_query()
        with self.__engine.connect() as con:
            con.execute(text(query))
            con.commit()

In [7]:
class MonthlyUpdater:
    def __init__(self):
        
        self.__get_engine()
        
    def __load_dotenv(self):
        load_dotenv()
        return {
            'user': os.getenv("DB_USER"),
            'password': os.getenv("DB_PASSWORD"),
            'host': 'localhost',
            'port': os.getenv("DB_PORT"),
            'db_name': os.getenv("DB_NAME"),
            'schema': 'fin'
        }

    def __get_engine(self):
        conf = self.__load_dotenv()
        self.__engine = create_engine(f"postgresql+psycopg2://{conf['user']}:{conf['password']}@{conf['host']}:{conf['port']}/{conf['db_name']}")

    def __read_sql_query(self, query):
        with self.__engine.begin() as con:
            return pd.read_sql_query(query, con)

    def update_table(self):
        # c_month = datetime.now().month 
        c_month = 9
        c_year = datetime.now().year
        query = f"""
            select * 
            from fin.revenue_est_2025 rev
            where date_part('month', rev.estimate_date) = {c_month - 1}
            and date_part('year', rev.estimate_date) = {c_year}
            and date_part('month', rev.date_dt) >= {c_month}
        """
        data = self.__read_sql_query(query)[['company', 'date_dt', 'estimate_date', 'frc', 'est_amount', 'hcl_amount', 'contr_amount']].assign(estimate_date = lambda x: x['estimate_date'] + relativedelta(months=1))
        with self.__engine.connect() as con:
            data.to_sql("revenue_est_2025", con, schema='fin', if_exists='append', index=False)

In [10]:
monthly_upd = MonthlyUpdater()
monthly_upd.update_table()

In [33]:
class YearUpdater:
    def __init__(self):
        
        self.__get_engine()
        
    def __load_dotenv(self):
        load_dotenv()
        return {
            'user': os.getenv("DB_USER"),
            'password': os.getenv("DB_PASSWORD"),
            'host': 'localhost',
            'port': os.getenv("DB_PORT"),
            'db_name': os.getenv("DB_NAME"),
            'schema': 'fin'
        }

    def __get_engine(self):
        conf = self.__load_dotenv()
        self.__engine = create_engine(f"postgresql+psycopg2://{conf['user']}:{conf['password']}@{conf['host']}:{conf['port']}/{conf['db_name']}")

    def __read_sql_query(self, query):
        with self.__engine.begin() as con:
            return pd.read_sql_query(query, con)

    def __get_all_frc(self):
        c_year = datetime.now().year
        query = f"""
            select rev.company, rev.frc 
            from fin.revenue_est_2025 rev
            group by rev.company, rev.frc
            """
        return self.__read_sql_query(query)
       

    def __get_all_dates(self):
        c_year = datetime.now().year
        return pd.DataFrame({'date_dt': [datetime(c_year, 1, 1).date() + relativedelta(months=i) for i in range(12)],
                            'estimate_date': [datetime(c_year, 1, 1).date()] * 12,
                            'est_amount': [None] * 12,
                            'hcl_amount': [None] * 12,
                            'contr_amount': [None] * 12})

    def update_table(self):
        company_frc = self.__get_all_frc()
        dates_amounts = self.__get_all_dates()
        result_df = company_frc.join(dates_amounts, how='cross')[['company', 'date_dt', 
                                                                  'estimate_date', 'frc',
                                                                 'est_amount', 'hcl_amount', 'contr_amount']]
        with self.__engine.connect() as con:
            result_df.to_sql("revenue_est_2025", con, schema='fin', if_exists='append', index=False)

In [34]:
year_updater = YearUpdater()
year_updater.update_table()